In [57]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [58]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [59]:
import tensorflow as tf

In [60]:
data_df= pd.read_csv("Combined_News_DJIA.csv")

In [61]:
data_df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [62]:
#df['period'] = df[['Year', 'quarter']].apply(lambda x: ''.join(x), axis=1)
'''
top_news_list= ['Top1','Top2','Top3','Top4','Top5','Top6','Top7','Top8','Top9','Top10',
             'Top11','Top12','Top13','Top14','Top15','Top16','Top17','Top18','Top19','Top20',
             'Top21','Top22','Top23','Top24','Top25']
'''

top_news_list= ['Top1','Top2','Top3','Top4','Top5','Top6','Top7','Top8','Top9','Top10',
               'Top11','Top12','Top13','Top14','Top15','Top16','Top17','Top18','Top19','Top20',
               'Top21','Top22']

data_df['TopNews'] = data_df[top_news_list].apply(lambda x: ','.join(x), axis=1)

# for some reasons above command is not concatenating Top23-Top25
data_df['TopNews']= data_df['TopNews']+','+ data_df['Top23']+','+data_df['Top24']+ ',' + data_df['Top25']

In [63]:
data_df['TopNews'][2]

'b\'Remember that adorable 9-year-old who sang at the opening ceremonies? That was fake, too.\',b"Russia \'ends Georgia operation\'",b\'"If we had no sexual harassment we would have no children..."\',b"Al-Qa\'eda is losing support in Iraq because of a brutal crackdown on activities it regards as un-Islamic - including women buying cucumbers",b\'Ceasefire in Georgia: Putin Outmaneuvers the West\',b\'Why Microsoft and Intel tried to kill the XO $100 laptop\',b\'Stratfor: The Russo-Georgian War and the Balance of Power   \',b"I\'m Trying to Get a Sense of This Whole Georgia-Russia War: Vote Up If You Think Georgia Started It, Or Down If you Think Russia Did",b"The US military was surprised by the timing and swiftness of the Russian military\'s move into South Ossetia and is still trying to sort out what happened, a US defense official said Monday",b\'U.S. Beats War Drum as Iran Dumps the Dollar\',b\'Gorbachev: "Georgian military attacked the South Ossetian capital of Tskhinvali with multi

In [64]:
merged_df= data_df[['Date','Label','TopNews']]

In [65]:
merged_df.head()

,Date,Label,TopNews
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou..."
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...
4,2008-08-14,1,b'All the experts admit that we should legalis...


In [66]:
def news_to_words (raw_news):
    
    # Remove the HTML characters
    news_text = BeautifulSoup(raw_news)
    
    # Remove the punctuals
    letters_only = re.sub("[^a-zA-Z0-9.]"," ",news_text.get_text())
    
    # Convert it in lower case and split them into words
    lower_case = letters_only.lower()
    words = lower_case.split()
    
    
    
    #In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english")) 
    
    meaningfull_words = [w for w in words if not w in stops]
    
    
    #Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningfull_words ))

In [67]:

clean_news= news_to_words(merged_df['TopNews'][0])
clean_news

/Users/rajegupt/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/rajegupt/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


'b georgia downs two russian warplanes countries move brink war b breaking musharraf impeached. b russia today columns troops roll south ossetia footage fighting youtube b russian tanks moving towards capital south ossetia reportedly completely destroyed georgian artillery fire b afghan children raped impunity u.n. official says sick three year old raped nothing b 150 russian tanks entered south ossetia whilst georgia shoots two russian jets. b breaking georgia invades south ossetia russia warned would intervene side b enemy combatent trials nothing sham salim haman sentenced 5 1 2 years kept longer anyway feel like it. b georgian troops retreat s. osettain capital presumably leaving several hundred people killed. video b u.s. prep georgia war russia b rice gives green light israel attack iran says u.s. veto israeli military ops b announcing class action lawsuit behalf american public fbi b russia georgia war nyt top story opening ceremonies olympics fucking disgrace yet proof decline 

In [68]:
num_newsdays= merged_df['TopNews'].size
num_newsdays

1989

In [69]:
clean_news_all= []

for i in range(0, num_newsdays):
    # If the index is evenly divisible by 400, print a message
    if( (i+1)%400 == 0 ):
        print ("Review %d of %d\n" % ( i+1, num_newsdays))
    
    clean_news_all.append(news_to_words(str(merged_df['TopNews'][i])))

/Users/rajegupt/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/rajegupt/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Review 400 of 1989

Review 800 of 1989

Review 1200 of 1989

Review 1600 of 1989



In [70]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

clean_news_all_1= [[porter_stemmer.stem(word) for word in sentence.split(" ")] for sentence in clean_news_all]
clean_news_all_1= [" ".join(sentence) for sentence in clean_news_all_1]

In [71]:
all_text = ''.join([c for c in clean_news_all_1])
reviews = all_text.split(' b ')

all_text = ' '.join(reviews)
words = all_text.split()

In [72]:
#all_text

In [73]:
words

['b',
 'georgia',
 'down',
 'two',
 'russian',
 'warplan',
 'countri',
 'move',
 'brink',
 'war',
 'break',
 'musharraf',
 'impeached.',
 'russia',
 'today',
 'column',
 'troop',
 'roll',
 'south',
 'ossetia',
 'footag',
 'fight',
 'youtub',
 'russian',
 'tank',
 'move',
 'toward',
 'capit',
 'south',
 'ossetia',
 'reportedli',
 'complet',
 'destroy',
 'georgian',
 'artilleri',
 'fire',
 'afghan',
 'children',
 'rape',
 'impun',
 'u.n.',
 'offici',
 'say',
 'sick',
 'three',
 'year',
 'old',
 'rape',
 'noth',
 '150',
 'russian',
 'tank',
 'enter',
 'south',
 'ossetia',
 'whilst',
 'georgia',
 'shoot',
 'two',
 'russian',
 'jets.',
 'break',
 'georgia',
 'invad',
 'south',
 'ossetia',
 'russia',
 'warn',
 'would',
 'interven',
 'side',
 'enemi',
 'combat',
 'trial',
 'noth',
 'sham',
 'salim',
 'haman',
 'sentenc',
 '5',
 '1',
 '2',
 'year',
 'kept',
 'longer',
 'anyway',
 'feel',
 'like',
 'it.',
 'georgian',
 'troop',
 'retreat',
 's.',
 'osettain',
 'capit',
 'presum',
 'leav',
 'sev

# Encoding the words

### create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

In [74]:
from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}



In [75]:
news_ints = []
for r in clean_news_all_1:
    ri = [vocab_to_int.get(w) for w in r.split(" ") if vocab_to_int.get(w) is not None]
    news_ints.append(ri)
news_ints[:10]

[[12254,
  943,
  2068,
  34,
  30,
  2168,
  14,
  244,
  1940,
  17,
  12254,
  383,
  6769,
  16144,
  12254,
  13,
  374,
  5928,
  187,
  2102,
  58,
  2722,
  1064,
  140,
  1559,
  12254,
  30,
  813,
  244,
  710,
  443,
  58,
  2722,
  935,
  926,
  312,
  2553,
  2353,
  76,
  12254,
  221,
  93,
  146,
  6770,
  540,
  36,
  1,
  2354,
  126,
  3,
  62,
  146,
  1187,
  12254,
  1399,
  30,
  813,
  515,
  58,
  2722,
  7341,
  943,
  431,
  34,
  30,
  16145,
  12254,
  383,
  943,
  1683,
  58,
  2722,
  13,
  72,
  78,
  2723,
  927,
  12254,
  1572,
  1132,
  411,
  1187,
  5346,
  12255,
  16146,
  217,
  288,
  98,
  154,
  3,
  1652,
  666,
  5607,
  1096,
  83,
  905,
  12254,
  2553,
  187,
  2962,
  1630,
  16147,
  443,
  6321,
  257,
  450,
  301,
  11,
  2641,
  155,
  12254,
  20,
  8955,
  943,
  17,
  13,
  12254,
  2642,
  173,
  885,
  949,
  8,
  16,
  31,
  1,
  20,
  2834,
  21,
  37,
  2554,
  12254,
  345,
  1037,
  414,
  1432,
  3315,
  99,
  135,
  

In [76]:
print (len(clean_news_all_1[1].split(" ")))
print (len(news_ints[1]))
print (len(news_ints))

198
198
1989


In [77]:
labels = np.array(merged_df['Label'])
print (len(labels))

1989


In [78]:
from collections import Counter
new_lens = Counter([len(x) for x in news_ints])
print("Zero-length news: {}".format(new_lens[0]))
print("Maximum new length: {}".format(max(new_lens)))

Zero-length news: 3
Maximum new length: 489


In [79]:
count=0
for x in news_ints:
    count+=1
    if len(x)==0:
        print (x, count)

## 271, 348, 681 have top 23,34,25 news as NaN..may need to fix it later

[] 278
[] 349
[] 682


In [80]:
# Filter out that review with 0 length
#Let's truncate to 400 steps. For news shorter than 400, 
#we'll pad with 0s. For news longer than 200, we can truncate them to the first 200 characters.
news_ints = [r[0:400] for r in news_ints if len(r) > 0]

In [81]:
from collections import Counter
new_lens = Counter([len(x) for x in news_ints])
print("Zero-length news: {}".format(new_lens[0]))
print("Maximum new length: {}".format(max(new_lens)))

Zero-length news: 0
Maximum new length: 400


In [82]:
# removing the corresponding labels
labels= np.delete(labels,[277,348,681])
len(labels)

1986

In [83]:
# Padding 0 for the news shorter than 400

seq_len = 400
features = np.zeros((len(news_ints), seq_len), dtype=int)
#print(features[:10,:100])
for i, row in enumerate(news_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
features[:10,100:200]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 12254,   943,  2068,
           34,    30,  2168,    14,   244,  1940,    17, 12254,   383,
         6769, 16144, 12254,    13,   374,  5928,   187,  2102,    58,
         2722,  1064,   140,  1559, 12254,    30,   813,   244,   710,
          443,    58,  2722,   935,   926,   312,  2553,  2353,    76,
        12254,   221,    93,   146,  6770,   540,    36,     1,  2354,
          126,     3,    62,   146,  1187, 12254,  1399,    30,   813,
          515,    58,  2722,  7341,   943,   431,    34,    30, 16145,
        12254,   383,   943,  1683,    58,  2722,    13,    72,    78,
         2723],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     

In [84]:
print ("feature shape", features.shape)
print  ("label shape", labels.shape)

feature shape (1986, 400)
label shape (1986,)


In [85]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size =0.2, random_state=0)

In [86]:
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nTest set: \t\t{}".format(test_x.shape))

print("label set: \t\t{}".format(train_y.shape), 
      "\nTest label set: \t\t{}".format(test_y.shape))

			Feature Shapes:
Train set: 		(1588, 400) 
Test set: 		(398, 400)
label set: 		(1588,) 
Test label set: 		(398,)


# Build the Graph

In [87]:
# Defining Hyperparams
lstm_size = 256
lstm_layers = 2
batch_size = 100
learning_rate = 0.01

In [88]:
n_words= len(vocab_to_int)+1
n_words

32105

In [89]:
n_words= len(vocab_to_int)+1 # add 1 for 0 added to vocab

# Create the graph object
tf.reset_default_graph()
with tf.name_scope('inputs'):
    inputs_ = tf.placeholder(tf.int32, [None, None], name = 'inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name = 'labels')
    keep_prob = tf.placeholder(tf.float32, name= 'keep_prob')
    

In [90]:
embed_size = 300

with tf.name_scope("Embeddings"):
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size),-1,1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

In [91]:
# tf.contrib.rnn.BasicLSTMCell to create an LSTM cell. 
# Then, add drop out to it with tf.contrib.rnn.DropoutWrapper. 
# Finally, create multiple LSTM layers with tf.contrib.rnn.MultiRNNCell.


def lstm_cell():
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(lstm_size, reuse=tf.get_variable_scope().reuse)
    # Add dropout to the cell
    return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)

with tf.name_scope("RNN_layers"):
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    

## RNN Forward pass

In [92]:
'''
Above I created an initial state, initial_state, to pass to the RNN. 
This is the cell state that is passed between the hidden layers in successive time steps. 
tf.nn.dynamic_rnn takes care of most of the work for us. 
We pass in our cell and the input to the cell, then it does the unrolling and everything else for us.
It returns outputs for each time step and the final_state of the hidden layer.

'''


with tf.name_scope("RNN_forward"):
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state=initial_state)

## Output

We only care about the final output, we'll be using that as our sentiment prediction. So we need to grab the last output with outputs[:, -1], the calculate the cost from that and labels_.

In [93]:
with tf.name_scope("Prediction"):
    predictions = tf.contrib.layers.fully_connected(outputs[:,-1], 1 , activation_fn= tf.sigmoid)
    tf.summary.histogram("predictions", predictions)
with tf.name_scope("cost"):
    cost = tf.losses.mean_squared_error(labels_, predictions)
    tf.summary.scalar('cost', cost)
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
merged = tf.summary.merge_all()

### Validation Accuracy

In [94]:
with tf.name_scope("validation"):
    correct_pred= tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### Batching

This is a simple function for returning batches from our data. First it removes data such that we only have full batches. Then it iterates through the x and y arrays and returns slices out of those arrays with size [batch_size]

In [95]:
def get_batches(x,y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x,y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0,len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]
        

### Training

In [96]:
epochs =10

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    train_writer = tf.summary.FileWriter('./logs/tb/train', sess.graph)
    test_writer = tf.summary.FileWriter('./logs/tb/test', sess.graph)
    iteration =1
    
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x,y)in enumerate(get_batches(train_x, train_y, batch_size),1):
            feed = {inputs_:x, labels_:y[:,None], keep_prob: 0.5, initial_state: state}
            summary, loss, state, _ = sess.run([merged, cost, final_state,optimizer], feed_dict=feed)
            
            train_writer.add_summary(summary, iteration)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))
            
            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(test_x, test_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    summary, batch_acc, val_state = sess.run([merged, accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
            
            test_writer.add_summary(summary, iteration)
            saver.save(sess,"checkpoints/stocknews_rg.ckpt")
    saver.save(sess,"checkpoints/stocknews_rg.ckpt")

Epoch: 0/10 Iteration: 5 Train loss: 0.288
Epoch: 0/10 Iteration: 10 Train loss: 0.281
Epoch: 0/10 Iteration: 15 Train loss: 0.247
Epoch: 1/10 Iteration: 20 Train loss: 0.246
Epoch: 1/10 Iteration: 25 Train loss: 0.275
Val acc: 0.470
Epoch: 1/10 Iteration: 30 Train loss: 0.217
Epoch: 2/10 Iteration: 35 Train loss: 0.224
Epoch: 2/10 Iteration: 40 Train loss: 0.089
Epoch: 2/10 Iteration: 45 Train loss: 0.127
Epoch: 3/10 Iteration: 50 Train loss: 0.102
Val acc: 0.513
Epoch: 3/10 Iteration: 55 Train loss: 0.066
Epoch: 3/10 Iteration: 60 Train loss: 0.139
Epoch: 4/10 Iteration: 65 Train loss: 0.029
Epoch: 4/10 Iteration: 70 Train loss: 0.025
Epoch: 4/10 Iteration: 75 Train loss: 0.026
Val acc: 0.500
Epoch: 5/10 Iteration: 80 Train loss: 0.009
Epoch: 5/10 Iteration: 85 Train loss: 0.020
Epoch: 5/10 Iteration: 90 Train loss: 0.017
Epoch: 6/10 Iteration: 95 Train loss: 0.014
Epoch: 6/10 Iteration: 100 Train loss: 0.000
Val acc: 0.497
Epoch: 6/10 Iteration: 105 Train loss: 0.010
Epoch: 7/10 Ite

In [97]:
test_acc=[]
with tf.Session() as sess:
    saver.restore(sess, "checkpoints/stocknews_rg.ckpt")
    test_state= sess.run(cell.zero_state(batch_size, tf.float32))
    
    for x, y in get_batches(test_x, test_y, batch_size):
        feed = {inputs_: x,
               labels_: y[:,None],
               keep_prob:1, 
               initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    
    print (" Test Accuracy: {:3f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from checkpoints/stocknews_rg.ckpt
 Test Accuracy: 0.496667
